In [1]:
import ruptures as rpt  # our package
from ruptures.metrics import hausdorff
import pandas as pd
import concurrent.futures
import sys
infile="/Users/irffanalahi/Research/Research_code/gitignorefolder/DMRrelated/ITGAE_patternrecognition/changepointtest/preproocess_Develop/preprocess_ontopmetilene/input_out_mincpg2_intersectedwith_paramA_short.txt_header/ITGAEp_cin_nr0.5_imputed_g1_CD8TIL_3_g2_others_36_default.txt"
minCpG=3
outfile=infile+"_mincpg_"+str(minCpG)+"pelt.txt"
penalty_value = 1 
percormetDMR=2 #### for parallal
indf=pd.read_csv(infile,sep="\t")

indf.head()

,chrom,start,end,CD8TIL-CD4bluourPBL,CD8TIL-CD8bluourPBL,CD8TIL-nB,CD8TIL-NK,CD8TIL-MnbluourPBL,CD8TIL-iDC,CD8TIL-mDC,...,DMRchrom,DMRstart,DMRend,q,diff,#cpg,p(MWU),p(2dks),CD8TIL,others
0,chr17,3800169,3800171,-0.460267,-0.369467,-0.520667,-0.571000,-0.600067,-0.563000,-0.604667,...,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755
1,chr17,3800340,3800342,-0.408067,-0.392067,-0.378000,-0.405333,-0.394267,-0.404333,-0.385333,...,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755
2,chr17,3800416,3800418,-0.380200,-0.279200,-0.376000,-0.319667,-0.355400,-0.339000,-0.291667,...,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755
3,chr17,3800453,3800455,-0.369400,-0.298200,-0.406333,-0.355333,-0.360000,-0.359667,-0.289333,...,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755
4,chr17,3800479,3800481,-0.245800,-0.246800,-0.250000,-0.251333,-0.262200,-0.165000,-0.218333,...,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755


In [2]:
indfdropped=indf.drop(['chrom','start','end'],axis=1)
columnnames=(indfdropped.columns).tolist()
indfdropped.head()

,CD8TIL-CD4bluourPBL,CD8TIL-CD8bluourPBL,CD8TIL-nB,CD8TIL-NK,CD8TIL-MnbluourPBL,CD8TIL-iDC,CD8TIL-mDC,CD8TIL-mNeu,CD8TIL-cB,CD8TIL-MEL_TUMOR,DMRchrom,DMRstart,DMRend,q,diff,#cpg,p(MWU),p(2dks),CD8TIL,others
0,-0.460267,-0.369467,-0.520667,-0.571000,-0.600067,-0.563000,-0.604667,-0.453333,-0.165167,0.000667,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755
1,-0.408067,-0.392067,-0.378000,-0.405333,-0.394267,-0.404333,-0.385333,-0.388667,-0.405000,-0.152000,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755
2,-0.380200,-0.279200,-0.376000,-0.319667,-0.355400,-0.339000,-0.291667,-0.318333,-0.293333,-0.184667,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755
3,-0.369400,-0.298200,-0.406333,-0.355333,-0.360000,-0.359667,-0.289333,-0.321000,-0.277667,-0.258667,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755
4,-0.245800,-0.246800,-0.250000,-0.251333,-0.262200,-0.165000,-0.218333,-0.233000,-0.192000,0.013667,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755


In [3]:
def segment_generator():
    
    segmentlist=[]
    
    qindex=columnnames.index('q')

    subdf=indfdropped.iloc[:,:qindex]
   
    grouped=subdf.groupby(['DMRchrom','DMRstart','DMRend'])
    for name, group in grouped:
        subgroup=group.drop(["DMRchrom","DMRstart","DMRend"],axis=1)
        segmentlist.append((name,subgroup))
    
  
    
    return segmentlist


    

In [4]:
class PELT_result:
    def __init__(self,dmrchrom,dmrstart,dmrend,peltsegmentlist):
        self.dmrchrom=dmrchrom
        self.dmrstart=dmrstart
        self.dmrend=dmrend
        self.peltsegmentlist=peltsegmentlist
        self.totalsegment=len(self.peltsegmentlist)-1
        self.metpeltsegmentDICT={'DMRchrom':self.dmrchrom,'DMRstart':self.dmrstart,'DMRend':self.dmrend,"totalPeltSegement":self.totalsegment}
        
        

In [5]:
def get_pelt_result(signal_pelttuple):
    DMR=signal_pelttuple[0]
    signal_pelt=signal_pelttuple[1]
    algo_python = rpt.Pelt(model="rbf", jump=1, min_size=minCpG).fit(signal_pelt)
    bkps_python = algo_python.predict(pen=penalty_value)
    forindexgenerate=bkps_python
    if forindexgenerate[-1]==len(signal_pelt.index):
        forindexgenerate[-1]=forindexgenerate[-1]-1
    pr=PELT_result(DMR[0],DMR[1],DMR[2],(signal_pelt.index[forindexgenerate]).tolist())
    return pr

def run_get_pelt_result(signalchunks):
    signalchunksresult=[]
    for signalchunk in signalchunks:
        signalchunksresult.append(get_pelt_result(signalchunk))
    return signalchunksresult
        

In [6]:
signallist=segment_generator()
chunks = [signallist[x:x+percormetDMR] for x in range(0, len(signallist), percormetDMR)]

In [7]:
multiresult = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    processlist=[]
    for chunk in chunks:
        processlist.append(executor.submit(run_get_pelt_result,chunk))
        #algo_python = rpt.Pelt(model="rbf", jump=1, min_size=minCpG).fit(signal)  # written in pure python
    for process in concurrent.futures.as_completed(processlist):
        multiresult=multiresult+process.result()

In [8]:
allresultdictlist=[]
for resultobj in multiresult:
    allresultdictlist.append(resultobj.metpeltsegmentDICT)
allresultdf=pd.DataFrame(allresultdictlist)

In [9]:
dmrchrindex=columnnames.index('DMRchrom')
foroutsubdf=indfdropped.iloc[:,dmrchrindex:]
foroutsubdf=foroutsubdf.drop_duplicates(keep='first')
foroutsubdf.head()

,DMRchrom,DMRstart,DMRend,q,diff,#cpg,p(MWU),p(2dks),CD8TIL,others
0,chr17,3800170,3801629,3.708200e-23,-0.350201,28,4.596300e-14,3.708200e-23,0.41735,0.76755


In [10]:
outdf=foroutsubdf.merge(allresultdf,how='inner',on=['DMRchrom','DMRstart','DMRend'])
outdf=outdf.rename(columns ={'DMRchrom':'chrom','DMRstart':'start','DMRend':'end'},errors='raise')
outdf.to_csv(outfile,sep="\t",index=False)